In [ ]:
# pip install pandas 

In [ ]:
import pickle
import time
from datetime import datetime, timedelta
from easydict import EasyDict as edict
import pandas as pd
from tqdm import tqdm
import os
import matplotlib.pyplot as plt

現貨

In [ ]:
symbol = 'BTCUSDT'
f_symbol = "BTCUSD_PERP"
# freq = '30m'
# freq = '4h'
# freq = '8h'
freq = '1d'

In [ ]:
all_klines_table = pd.read_pickle(f"{symbol}_{freq}_spot_k_line_data.pkl")
all_f_klines_table = pd.read_pickle(f"{symbol}_{freq}_future_k_line_data.pkl")

# 合併資料

In [ ]:
merge_all_klines_table = all_klines_table[['open_time', 'close', 'open']].merge(
    all_f_klines_table[['open_time', 'close', 'open']], on=['open_time'], 
    suffixes=("_spot", "_future"),
)

In [ ]:
merge_all_klines_table['basis'] =merge_all_klines_table['close_spot'] - merge_all_klines_table['close_future']
merge_all_klines_table['basis_rate'] = merge_all_klines_table['basis']/merge_all_klines_table['close_spot'] * 100
merge_all_klines_table.index = merge_all_klines_table['open_time']

# 我們想探討的變數

In [ ]:
y = merge_all_klines_table["basis"]

# 畫圖

In [ ]:
outpath = "output"
os.makedirs(outpath,exist_ok=True)

In [ ]:
plt.figure(figsize=(10,4))
title = f"{symbol}-{freq}-{y.name}"
plt.title(title, fontdict={'fontname': 'Times New Roman', 'fontstyle': 'italic'})
plt.plot(y)
plt.grid()
plt.tight_layout()
plt.savefig(os.path.join(outpath, f"{title}走勢圖.png"))

In [ ]:
plt.figure(figsize=(10,4))
n, bins, patches = plt.hist(y, bins=100, alpha=0.75, edgecolor='black')
title = f"{symbol}-{freq}-{y.name}"
plt.title(title, fontdict={'fontname': 'Times New Roman', 'fontstyle': 'italic'})
plt.grid()
plt.tight_layout()
plt.savefig(os.path.join(outpath, f"{title}分佈圖.png"))

# 敘述性統計

In [ ]:
# !pip install arch
# !pip install numpy==1.23.5 --user

In [ ]:
from utils import describeData

In [ ]:
file_name = f"{symbol}-{freq}-{y.name}"
describeData(y).to_frame().to_csv(os.path.join(outpath, f'{file_name}_describe.csv') )
describeData(y)

In [ ]:
# 單根檢定

In [ ]:
title

In [ ]:
from arch import unitroot
adf = unitroot.ADF(y, max_lags=50, method="bic")
print(adf.summary().as_text())

In [ ]:
pp = unitroot.PhillipsPerron(y)
print(pp.summary().as_text())

In [ ]:
kpss = unitroot.KPSS(y)
print(kpss.summary().as_text())

# acf, pacf

In [ ]:
import statsmodels.api as sm
from statsmodels.tsa.stattools import acf, pacf

In [ ]:
title = f"{symbol}-{freq}-{y.name}"
fig = plt.figure(figsize=(12, 8))
# plt.title(title, fontdict={'fontname': 'Times New Roman', 'fontstyle': 'italic'})
ax1 = fig.add_subplot(211)
fig = sm.graphics.tsa.plot_acf(y, lags=40, ax=ax1, title=f"{title}- ACF")
plt.grid()
plt.tight_layout()
ax2 = fig.add_subplot(212)
fig = sm.graphics.tsa.plot_pacf(y, lags=40, ax=ax2, title=f"{title}- PACF")
plt.grid()
plt.tight_layout()
plt.savefig(os.path.join(outpath, f"{title}_acf_pacf_分佈圖.png"))

In [ ]:
## 連第0期也有呈現
## if qstat=True, returns the Ljung-Box q statistic (coefficient, q_stat, prob)
coefficient, q_stat, prob = acf(y, nlags=40, qstat=True)

In [ ]:
pd.DataFrame([coefficient, q_stat, prob ], index=["acf", 'q', 'prob']).T

In [ ]:
pacf_value, pacf_c = pacf(y,  nlags=40, alpha=.05)

In [ ]:
pd.DataFrame([pacf_value, pacf_c ], index=["pacf ", "confint"], columns=["" for _ in range(len(pacf_value))]).T

https://www.statsmodels.org/devel/examples/notebooks/generated/tsa_arma_0.html

In [ ]:
import pandas as pd
import numpy as np 
np.set_printoptions( precision= 5, suppress=True )

import matplotlib.pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import acf, pacf

from statsmodels.tsa.arima.model import ARIMA
from statsmodels.graphics.api import qqplot
import statsmodels.api as sm
import numpy as np
import pandas as pd
from statsmodels.stats.diagnostic import acorr_ljungbox
from statsmodels.graphics.tsaplots import plot_predict

In [ ]:
ARIMA?

In [ ]:
p = 1
q = 5

# ARIMA(p, d, q)
arma_model = ARIMA(
    y , 
    order=(p, 0, q), 
    missing='drop',
    # freq=freq
).fit()

In [ ]:
summary = arma_model.summary()
# 殘差檢驗：Ljung-Box統計
Q_test_table = acorr_ljungbox(arma_model.resid, lags=[4, 8, 12], return_df=False)

In [ ]:
# 計算 SSR
ssr = np.sum(arma_model.resid**2)

# R 平方
r_squared = 1 - ssr/np.sum((y - np.mean(y))**2)

In [ ]:
summary

In [ ]:
print(f"SSR: {ssr}")
print(f"R^2: {r_squared}")

In [ ]:
Q_test_table

In [ ]:
from tqdm import tqdm

In [ ]:
def signif(p_val):
    if p_val <= 0.01: return "***"
    elif p_val <= 0.05: return "**"
    elif p_val <= 0.1: return "*"
    else: return ""


In [ ]:
all_result_table = pd.DataFrame()
resid_maplist = {}
for pq in tqdm([
    (1,0), 
    (3,0), 
    ((1,7),0), 
    ((1,8),0), 
    ((1,7,8),0), 
    ((1,2,3,7),0),
    ((1,2,3,8),0),
    ((1,2,3,7,8),0),

    (0,1),
    (0,5), 
    (1,1), 
    (1,5), 
    (3,1), 
    ((1,2,3,7,8),1),  
    (3,5), 
    ((1,2,3,7,8),5),  
]):
    p = pq[0]
    q = pq[1]
    
    # ARIMA(p, d, q)
    try:
        arma_model = ARIMA(
            y , 
            order=(p, 0, q), 
            missing='drop',
            # freq=freq
        ).fit()
    except:
        continue
    summary = arma_model.summary()
    # 殘差檢驗：Ljung-Box統計
    Q_test_table = acorr_ljungbox(arma_model.resid, lags=[4, 8, 12], return_df=False)
    # 計算 SSR
    ssr = np.sum(arma_model.resid**2)
    
    # R 平方
    r_squared = 1 - ssr/np.sum((y - np.mean(y))**2)

    result_label_list = []
    result_value_list = []
    for res_coef, res_t,res_p, res_name in zip(arma_model.params , arma_model.tvalues, arma_model.pvalues, arma_model.params.index):
        result_label_list.append(f"{res_name}_coef")
        result_value_list.append("%.4f%s"%(res_coef, signif(res_p)))
        result_label_list.append(f"{res_name}_t")
        result_value_list.append("%.4f"%res_t)

    result_label_list.append("R-sqr")
    result_value_list.append("%.4f"% r_squared)
    
    result_label_list.append("SSR")
    result_value_list.append("%.4f"% ssr)
    
    result_label_list.append("AIC")
    result_value_list.append("%.4f"%arma_model.aic)
        
    result_label_list.append("SBC")
    result_value_list.append("%.4f"%arma_model.bic)
    
    result_label_list.append("Q(4)")
    result_value_list.append("%.4f"%Q_test_table['lb_stat'][4])
    
    result_label_list.append("Q(8)")
    result_value_list.append("%.4f"%Q_test_table['lb_stat'][8])
    
    result_label_list.append("Q(12)")
    result_value_list.append("%.4f"%Q_test_table['lb_stat'][12])
    
    res_table =  pd.DataFrame(result_value_list, index=result_label_list, columns=[f"ARMA({p},{q})"])
    
    all_result_table = pd.concat((all_result_table, res_table), axis=1)
    resid_maplist[f"ARMA({p},{q})"] = arma_model.resid

In [ ]:
arma_model.summary()

In [ ]:
title = f"{symbol}-{freq}-{y.name}"

In [ ]:
all_result_table.loc[result_label_list, :].to_excel(os.path.join(outpath, f"{title}_arma_result.xlsx"))
pd.to_pickle(resid_maplist, os.path.join(outpath, f"{title}_arma_result_resid.pkl"))

In [ ]:
all_result_table.loc[result_label_list, :]

# 殘差診斷

In [ ]:
p = (1, 2, 3, 7, 8)
g = 5
arma_model_resid = resid_maplist[f"ARMA({p},{q})"] 

In [ ]:
title = f"{symbol}-{freq}-{y.name}-ARMA({p},{q})"
fig = plt.figure(figsize=(12, 8))
# plt.title(title, fontdict={'fontname': 'Times New Roman', 'fontstyle': 'italic'})
ax1 = fig.add_subplot(211)
fig = sm.graphics.tsa.plot_acf(arma_model_resid, lags=40, ax=ax1, title=f"{title}- ACF")
plt.grid()
plt.tight_layout()
ax2 = fig.add_subplot(212)
fig = sm.graphics.tsa.plot_pacf(arma_model_resid, lags=40, ax=ax2, title=f"{title}- PACF")
plt.grid()
plt.tight_layout()
plt.savefig(os.path.join(outpath, f"{title}_acf_pacf_分佈圖.png"))

In [ ]:
from statsmodels.stats.diagnostic import acorr_ljungbox
white_noise = acorr_ljungbox(arma_model_resid, lags = [4, 8, 12, 40], return_df=True)

In [ ]:
white_noise

# 預測

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(16, 8))
fig = plot_predict(arma_model, start="2024-03-25", end="2024-04-02", ax=ax)
legend = ax.legend(loc="upper left")